In [1]:
from typing import Union

import numpy as np
import matplotlib.pyplot as plt

## Simulation of Markov process

In [2]:
def sample_exp(lamb, n=1):
    U = np.random.rand(n)
    return - np.log(U) / lamb


def sample_class(p: Union[list, tuple, np.array]=None, n=1):
    if p is None:
        p = [0.5, 0.5]
    if isinstance(p, (list, tuple)):
        p = np.array(p)

    if np.abs(np.sum(p) - 1) > 1e-10:
        raise ValueError("Probabilities should add up to 1.")
        
    cum_probs = np.cumsum(p).reshape(p.size, 1)
    U = np.random.rand(1, n)
    return np.argmax(U < cum_probs, axis=0)

In [3]:
def markov_chain(x0, P, lambdas, max_iters=100):
    if isinstance(lambdas, (list, tuple)):
        lambdas = np.array(lambdas)
    if isinstance(P, (list, tuple)):
        P = np.array(P)

    x = [x0]
    
    T = [0]

    for _ in range(max_iters):
        current_x = x[-1]
        epsilon = float(sample_exp(lambdas[current_x])[0])
        
        T.append(T[-1] + epsilon)
        x.append(int(sample_class(P[current_x])[0]))
    return x, T

In [4]:
lamb = 4
mu = 2

N = 4

upper_diag = (N - 1) * [lamb]
main_diag = [-lamb] + [- (lamb + mu)] + (N - 3) * [- (lamb + 2 * mu)] + [-2*mu]
lower_diag = [mu] + (N - 2) * [2 * mu]

Q = np.diag(upper_diag, k=1) + np.diag(main_diag) + np.diag(lower_diag, k=-1)

lambdas = np.abs(np.diagonal(Q))
P = (Q - np.diag(np.diagonal(Q))) / lambdas.reshape(N, 1)

x, T = markov_chain(0, P, lambdas)

print(x)
print(T)

[0, 1, 2, 3, 2, 1, 0, 1, 0, 1, 0, 1, 2, 3, 2, 1, 0, 1, 0, 1, 0, 1, 0, 1, 2, 1, 2, 3, 2, 3, 2, 1, 2, 3, 2, 1, 2, 3, 2, 3, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 2, 3, 2, 1, 2, 1, 0, 1, 2, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 2, 1, 0, 1, 2, 3, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 2, 3, 2, 1, 2, 3, 2, 3, 2, 3, 2, 3, 2, 1, 2]
[0, 0.11684551675228931, 0.1624321213963319, 0.33692258738878655, 0.3378003208408333, 0.36654547685045685, 0.4414289092741934, 0.6354710607140239, 1.1654560939409615, 1.7650938671511223, 2.0382696857767844, 2.710319239381925, 2.839031698230828, 2.855329655229681, 3.0833122284699135, 3.199094307194681, 3.287605021022749, 3.487303995448828, 3.641472484882452, 3.6487116692117265, 4.099961165176155, 4.695755754699563, 4.755691362885048, 4.8184347647644055, 5.001072730717425, 5.262312171628037, 5.357239606362062, 5.740402790218562, 5.909014274494359, 6.050913619728192, 6.055987032602463, 6.453487769662142, 6.517732655440506, 6.558780448747115, 6.883796982916264, 6.907582484834263, 6.979971250